In [ ]:
!wget http://cvgl.stanford.edu/data/ActivityDataset.zip

--2024-04-04 05:52:01--  http://cvgl.stanford.edu/data/ActivityDataset.zip
Resolving cvgl.stanford.edu (cvgl.stanford.edu)... 171.64.64.64
Connecting to cvgl.stanford.edu (cvgl.stanford.edu)|171.64.64.64|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cvgl.stanford.edu/data/ActivityDataset.zip [following]
--2024-04-04 05:52:01--  https://cvgl.stanford.edu/data/ActivityDataset.zip
Connecting to cvgl.stanford.edu (cvgl.stanford.edu)|171.64.64.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1026546376 (979M) [application/zip]
Saving to: ‘ActivityDataset.zip’

ActivityDataset.zip 100%[===================>] 978.99M  13.4MB/s    in 77s     

2024-04-04 05:53:18 (12.7 MB/s) - ‘ActivityDataset.zip’ saved [1026546376/1026546376]



In [ ]:
!mkdir -p output/images
!mkdir -p output/lables

In [ ]:
!unzip ActivityDataset.zip -d input

Streaming output truncated to the last 5000 lines.
  inflating: input/seq37/frame0626.jpg  
  inflating: input/seq37/frame0627.jpg  
  inflating: input/seq37/frame0628.jpg  
  inflating: input/seq37/frame0629.jpg  
  inflating: input/seq37/frame0630.jpg  
  inflating: input/seq37/frame0631.jpg  
  inflating: input/seq37/frame0632.jpg  
  inflating: input/seq37/frame0633.jpg  
  inflating: input/seq37/frame0634.jpg  
  inflating: input/seq37/frame0635.jpg  
  inflating: input/seq37/frame0636.jpg  
  inflating: input/seq37/frame0637.jpg  
  inflating: input/seq37/frame0638.jpg  
  inflating: input/seq37/frame0639.jpg  
  inflating: input/seq37/frame0640.jpg  
  inflating: input/seq37/frame0641.jpg  
  inflating: input/seq37/frame0642.jpg  
  inflating: input/seq37/frame0643.jpg  
  inflating: input/seq37/frame0644.jpg  
  inflating: input/seq37/frame0645.jpg  
  inflating: input/seq37/frame0646.jpg  
  inflating: input/seq37/frame0647.jpg  
  inflating: input/seq37/frame0648.jpg  
  infl

In [ ]:
#####content of data.yaml
train: /content/yolo_output/train/images
val: /content/yolo_output/val/images

nc: 6  # number of classes

In [ ]:
import os
import shutil
from PIL import Image

# Function to convert annotation to YOLO format
def convert_to_yolo(x, y, width, height, image_width, image_height):
    x_center = (x + width / 2) / image_width
    y_center = (y + height / 2) / image_height
    width_normalized = width / image_width
    height_normalized = height / image_height
    return f"{x_center} {y_center} {width_normalized} {height_normalized}"

# Input and output directories
input_directory = '/content/input'
output_directory = '/content/output'

# Traverse each seqXX directory in the input directory
for seq_dir in os.listdir(input_directory):
    seq_path = os.path.join(input_directory, seq_dir)
    if os.path.isdir(seq_path):
        # Read annotation.txt
        annotation_file = os.path.join(seq_path, 'annotations.txt')
        with open(annotation_file, 'r') as f:
            lines = f.readlines()

        # Create corresponding labels directory if it doesn't exist
        output_labels_dir = os.path.join(output_directory, 'labels')
        os.makedirs(output_labels_dir, exist_ok=True)

        # Create corresponding images directory if it doesn't exist
        output_images_dir = os.path.join(output_directory, 'images')
        os.makedirs(output_images_dir, exist_ok=True)

        # Iterate through each line in annotation.txt
        for line in lines:
            parts = line.strip().split()
            frame_number = parts[0].zfill(4)  # Ensure 4-digit frame number
            x, y, width, height = map(int, parts[1:5])
            class_id = int(parts[5]) - 1  # Decrease class_id by 1

            # Get image width and height
            image_path = os.path.join(seq_path, f"frame{frame_number}.jpg")
            with Image.open(image_path) as img:
                image_width, image_height = img.size

            # Convert annotation to YOLO format
            yolo_annotation = convert_to_yolo(x, y, width, height, image_width, image_height)

            # Write YOLO annotation to a text file
            output_label_file = os.path.join(output_labels_dir, f"{seq_dir}frame{frame_number}.txt")
            with open(output_label_file, 'a') as f:
                f.write(f"{class_id} {yolo_annotation}\n")

            # Copy corresponding image frame to output/images directory
            output_image_file = os.path.join(output_images_dir, f"{seq_dir}frame{frame_number}.jpg")
            shutil.copy(image_path, output_image_file)

print("Conversion completed!")

Conversion completed!


In [ ]:
!mkdir -p yolo_output/train/images
!mkdir -p yolo_output/train/labels
!mkdir -p yolo_output/val/images
!mkdir -p yolo_output/val/labels

In [ ]:
!mv /content/output/images/seq{05,06,07,08,09,10,11,15,16,25,28,29}frame*.jpg yolo_output/val/images/
!mv /content/output/labels/seq{05,06,07,08,09,10,11,15,16,25,28,29}frame*.txt yolo_output/val/labels/

# Move remaining files for training set
!mv /content/output/images/*.jpg yolo_output/train/images/
!mv /content/output/labels/*.txt yolo_output/train/labels/

In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 749.1/749.1 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 85.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov9c.yaml')  # build a new model from YAML

# Train the model
results = model.train(data='data.yaml', epochs=100, imgsz=640)

Ultralytics YOLOv8.1.42 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov9c.yaml, data=data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, li

train: Scanning /content/yolo_output/train/labels.cache... 2576 images, 0 backgrounds, 8 corrupt: 100%|██████████| 2576/2576 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/yolo_output/train/images/seq21frame0171.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0644]
train: WARNING ⚠️ /content/yolo_output/train/images/seq22frame0141.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.2511]
train: WARNING ⚠️ /content/yolo_output/train/images/seq38frame0101.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      1.025]
train: WARNING ⚠️ /content/yolo_output/train/images/seq38frame0111.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0688]
train: WARNING ⚠️ /content/yolo_output/train/images/seq38frame0121.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0938]
train: WARNING ⚠️ /content/yolo_output/train/images/seq40frame0031.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0208]
train: WARNING ⚠️ /content/yolo_output/t

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/yolo_output/val/labels.cache... 890 images, 0 backgrounds, 2 corrupt: 100%|██████████| 890/890 [00:00<?, ?it/s]

...

    100/100      11.1G      0.446     0.2235     0.9563         45        640: 100%|██████████| 161/161 [02:04<00:00,  1.30it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 28/28 [00:13<00:00,  2.13it/s]

                   all        888       4146      0.555      0.624      0.571      0.473



100 epochs completed in 3.858 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 51.6MB
Optimizer stripped from runs/detect/train/weights/best.pt, 51.6MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.1.42 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv9c summary (fused): 384 layers, 25323874 parameters, 0 gradients, 102.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 28/28 [00:18<00:00,  1.54it/s]


                   all        888       4146      0.624      0.628      0.634      0.525
               class_0        888        242          0          0    0.00578    0.00377
               class_1        888        687      0.647      0.641      0.713      0.572
               class_2        888        583      0.814      0.585      0.717      0.627
               class_3        888        477      0.947      0.868       0.96      0.759
               class_4        888       1375      0.565        0.7      0.606       0.48
               class_5        888        782      0.771      0.974      0.806       0.71
Speed: 0.2ms preprocess, 10.1ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs/detect/train


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!zip -r yoloV9_runs.zip runs/

  adding: runs/ (stored 0%)
  adding: runs/detect/ (stored 0%)
  adding: runs/detect/train/ (stored 0%)
  adding: runs/detect/train/train_batch1.jpg (deflated 5%)
  adding: runs/detect/train/P_curve.png (deflated 8%)
  adding: runs/detect/train/events.out.tfevents.1712211401.b773fe34189c.6921.0 (deflated 91%)
  adding: runs/detect/train/val_batch0_pred.jpg (deflated 7%)
  adding: runs/detect/train/labels_correlogram.jpg (deflated 34%)
  adding: runs/detect/train/val_batch1_pred.jpg (deflated 10%)
  adding: runs/detect/train/F1_curve.png (deflated 9%)
  adding: runs/detect/train/train_batch0.jpg (deflated 4%)
  adding: runs/detect/train/weights/ (stored 0%)
  adding: runs/detect/train/weights/last.pt (deflated 9%)
  adding: runs/detect/train/weights/best.pt (deflated 9%)
  adding: runs/detect/train/results.png (deflated 8%)
  adding: runs/detect/train/train_batch14491.jpg (deflated 15%)
  adding: runs/detect/train/results.csv (deflated 85%)
  adding: runs/detect/train/args.yaml (deflate

In [ ]:
!cp yoloV9_runs.zip /content/drive/MyDrive

In [ ]:
!cp -r /content/runs /content/drive/MyDrive/yolov9_runs

In [ ]:
!rm -rf runs

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch

# Manually release memory
torch.cuda.empty_cache()